In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
import zipfile

from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin

In [3]:
# Step 1: Download the data.
url = 'http://mattmahoney.net/dc/'


def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    print('Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename
filename = maybe_download('text8.zip', 31344016)

3194880
Failed to verify text8.zip. Can you get to it with a browser?


In [4]:
# Read the data into a list of strings.
def read_data(filename):
  """Extract the first file enclosed in a zip file as a list of words"""
  with zipfile.ZipFile(filename) as f:
    data = tf.compat.as_str(f.read(f.namelist()[0])).split()
  return data

words = read_data('/Users/najeebkhan/Downloads/text8 (1).zip')
print('Data size', len(words))

Data size 17005207


In [5]:
# Step 2: Build the dictionary and replace rare words with UNK token.
vocabulary_size = 50000


def build_dataset(words, vocabulary_size):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words, vocabulary_size)
del words  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [6]:
data_index = 0
# Step 3: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels


In [7]:
batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3084 originated -> 12 as
3084 originated -> 5239 anarchism
12 as -> 3084 originated
12 as -> 6 a
6 a -> 12 as
6 a -> 195 term
195 term -> 6 a
195 term -> 2 of


In [8]:
print(batch)

[3084 3084   12   12    6    6  195  195]


### Creating the model

In [9]:
from tensorflow.contrib.tensorboard.plugins import projector
tf.reset_default_graph()

In [18]:
class SkipGram(object):
    
    def __init__(self,params):
        self.BATCH_SIZE = params['BATCH_SIZE']
        self.VOCAB_SIZE = params['VOCAB_SIZE']
        self.EMBED_SIZE = params['EMBED_SIZE']
        self.NUM_SAMPLED = params['NUM_SAMPLED']
        self.LEARNING_RATE =params['LEARNING_RATE']
        self.NUM_EPOCHS = params['NUM_EPOCHS']
    
    def _create_placeholders(self):
        self.center_words = tf.placeholder(tf.int32,shape=[self.BATCH_SIZE],name='center_words')
        self.target_words = tf.placeholder(tf.int32,shape=[self.BATCH_SIZE,1],name='target_words')
        
    def _create_embeddings(self):
        self.embeddings = tf.get_variable(name="embeddings",shape=[self.VOCAB_SIZE,self.EMBED_SIZE],
                                         initializer=tf.contrib.layers.xavier_initializer())
        self.embed = tf.nn.embedding_lookup(params=embeddings,ids=center_words)
    
    def _create_loss(self):
        self.nce_weight = tf.get_variable(name="nce_weight",shape=[self.VOCAB_SIZE,self.EMBED_SIZE],
                                         initializer=tf.contrib.layers.xavier_initializer())
        self.nce_bias = tf.Variable(tf.zeros([self.VOCAB_SIZE]),name="nce_bias")
        self.loss = tf.reduce_mean(tf.nn.nce_loss(weights=self.nce_weight,biases=self.nce_bias,
                                                  labels=self.target_words,inputs=self.embed,
                                                  num_sampled=self.NUM_SAMPLED,num_classes=self.VOCAB_SIZE))
        
    def _create_optimizer(self):
        self.optimizer = tf.train.GradientDescentOptimizer(self.LEARNING_RATE).minimize(self.loss)
        
    def train_model(self):
        init = tf.global_variables_initializer()
        with tf.Session() as sess:
            sess.run(init)
            writer = tf.summary.FileWriter('graphs/Embeddings/',sess.graph)
            self.epoch_loss = []
            for epoch in range(1,self.NUM_EPOCHS+1):
                batch,labels = generate_batch(batch_size=self.BATCH_SIZE,num_skips=2,skip_window=1)
                _,l = sess.run([self.optimizer,self.loss],feed_dict={self.center_words:batch,self.target_words:labels})
                if epoch%1000 == 0:
                    print('Epoch: {}\tLoss: {}'.format(epoch,l))
                self.epoch_loss.append(l)
    
    def _plot_loss(self):
        plt.plot(self.epoch_loss)
        plt.xlabel('Number of Epochs')
        plt.ylabel('Loss')
        plt.title('Loss of SkipGram Model')
        plt.plot()
        
    def _plot_embeddings(self):
        final_embed_matrix = sess.run(self.embeddings)
        embedding_var = tf.Variable(final_embed_matrix[:500], name='embedding')
        sess.run(embedding_var.initializer)
        config = projector.ProjectorConfig()
        summary_writer = tf.summary.FileWriter('graphs/Embeddings/')
        embedding = config.embeddings.add()
        embedding.tensor_name = embedding_var.name
        projector.visualize_embeddings(summary_writer, config)
        saver_embed = tf.train.Saver([embedding_var])
        saver_embed.save(sess,'graphs/Embeddings/skip-gram.ckpt', 1)

In [19]:
params = {'BATCH_SIZE':128,
          'VOCAB_SIZE':50000,
          'EMBED_SIZE':128,
          'NUM_SAMPLED':64,
          'LEARNING_RATE':1.0,
          'NUM_EPOCHS':10000}

In [20]:
skipgram = SkipGram(params=params)

In [21]:
with tf.name_scope("Data"):
    skipgram._create_placeholders()

In [22]:
with tf.name_scope("Embeddings"):
    skipgram._create_embeddings

In [23]:
with tf.name_scope("Loss"):
    skipgram._create_loss

In [24]:
skipgram._create_optimizer

<bound method SkipGram._create_optimizer of <__main__.SkipGram object at 0x128dd1890>>

In [25]:
skipgram.train_model()

AttributeError: 'SkipGram' object has no attribute 'optimizer'

In [ ]:
skipgram._plot_loss()